# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
!pip install transformers
!pip install peft
!pip install datasets
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 58.4 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 30.6 MB/s eta 0:00:00
Defaulting to user installation because normal site-packages is not writeable


#### Imports

In [5]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EvalPrediction
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from transformers import DataCollatorWithPadding
from peft import LoraConfig, PeftModelForSequenceClassification, TaskType, AutoPeftModelForSequenceClassification
import torch
import tqdm
from datasets import load_dataset

In [7]:
dataset = load_dataset('imdb')

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

#### Load tokenizer model and create training and testig datasets

In [86]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# tokenize dataset
def tokenize_batch(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

tokenized_dataset = dataset.map(tokenize_batch, batched=True)

train_dataset = tokenized_dataset['train'].shuffle(seed=42).select(range(500))
eval_dataset = tokenized_dataset['test'].shuffle(seed=42).select(range(500))

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [87]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)
model.config.pad_token_id = tokenizer.pad_token_id

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [89]:
# define compute matrics
def compute_matrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {"accuracy": accuracy_score(labels, predictions), "f1": f1, "precision": precision, "recall": recall}

In [90]:
# define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=100,
    warmup_ratio=0.1,
)

In [91]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_matrics,
    tokenizer=tokenizer
)

In [92]:
# start evaluation on foundation model
evaluation_results = trainer.evaluate()

print("Evaluation Results - ", evaluation_results)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Evaluation Results -  {'eval_loss': 0.6831517815589905, 'eval_accuracy': 0.612, 'eval_f1': 0.6117202774796136, 'eval_precision': 0.6119306625577813, 'eval_recall': 0.612, 'eval_runtime': 16.5644, 'eval_samples_per_second': 30.185, 'eval_steps_per_second': 3.803}


In [93]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [94]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=4,
    lora_alpha=16,
    lora_dropout=0.1
)

In [95]:
# Load pretrained model

peft_model = PeftModelForSequenceClassification(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 150,532 || all params: 108,460,804 || trainable%: 0.13878930862433952


In [96]:
# define compute matrics
def compute_matrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {"accuracy": accuracy_score(labels, predictions), "f1": f1, "precision": precision, "recall": recall}

In [102]:
# define training arguments
training_args = TrainingArguments(
    output_dir="./results/peft_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs/peft_model',
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=100,
    warmup_ratio=0.1,
)

In [103]:
# Initialize trainer
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_matrics,
    tokenizer=tokenizer
)

In [104]:
torch.cuda.empty_cache() # PyTorch thing

In [105]:
# Start trainig
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.678676,0.598000,0.566268,0.631227,0.598000
2,0.690600,0.675930,0.656000,0.655089,0.656732,0.656000
3,0.690600,0.672841,0.656000,0.655934,0.656624,0.656000
4,0.681500,0.670895,0.660000,0.659363,0.662314,0.660000
5,0.676000,0.670233,0.666000,0.665314,0.668535,0.666000


TrainOutput(global_step=315, training_loss=0.6820608229864211, metrics={'train_runtime': 261.396, 'train_samples_per_second': 9.564, 'train_steps_per_second': 1.205, 'total_flos': 658921912320000.0, 'train_loss': 0.6820608229864211, 'epoch': 5.0})

In [106]:
# Evaluate
evaluation_results = trainer.evaluate()

print("Evaluation Results - ", evaluation_results)

Evaluation Results -  {'eval_loss': 0.6702329516410828, 'eval_accuracy': 0.666, 'eval_f1': 0.665313640897456, 'eval_precision': 0.6685348308079585, 'eval_recall': 0.666, 'eval_runtime': 16.259, 'eval_samples_per_second': 30.752, 'eval_steps_per_second': 3.875, 'epoch': 5.0}


In [107]:
peft_model.save_pretrained('model/peft_model')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [108]:
inference_peft_model = AutoPeftModelForSequenceClassification.from_pretrained(
    "model/peft_model",
    num_labels=2
)
inference_peft_model.config.pad_token_id = inference_peft_model.config.eos_token_id

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [109]:
# Initialize trainer
trainer = Trainer(
    model=inference_peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_matrics,
    tokenizer=tokenizer
)

In [110]:
# Evaluate the model
evaluation_results = trainer.evaluate()
print("Evaluation Results:", evaluation_results)

Evaluation Results: {'eval_loss': 0.6702329516410828, 'eval_accuracy': 0.666, 'eval_f1': 0.665313640897456, 'eval_precision': 0.6685348308079585, 'eval_recall': 0.666, 'eval_runtime': 16.3157, 'eval_samples_per_second': 30.645, 'eval_steps_per_second': 3.861}


In [111]:
# get individual prediction
def predict_sentiment(review_sentence):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inference_peft_model.to(device)
    
    # prepare inputs
    inputs = tokenizer(review_sentence, return_tensors='pt').to(device)
    
    # get predictions 
    with torch.no_grad():
        outputs = inference_peft_model(**inputs)
        logits = outputs.logits
    
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    predicted_class_id = probabilities.argmax().item()

    return predicted_class_id

In [112]:
# Example usage
sentence = "The movie was so bad that I was about to come out on the interval and go watch Baby's day out!"
predicted_label = predict_sentiment(sentence)
print(f"Sentence: '{sentence}'\nPredicted label: {predicted_label}")

Sentence: 'The movie was so bad that I was about to come out on the interval and go watch Baby's day out!'
Predicted label: 0


In [113]:
# Example usage
sentence = "I recognized the Angry sister; the Runaway sister and the sister in Denial. I recognized the Abusive Husband and why he was there and then the Father, oh oh the Father... all superbly played. I also recognized myself and this movie was an eye-opener, a relief, a chance to face my OWN truth and finally doing something about it. I truly hope A Thousand Acres has had the same effect on some others out there."
predicted_label = predict_sentiment(sentence)
print(f"Sentence: '{sentence}'\nPredicted label: {predicted_label}")

Sentence: 'I recognized the Angry sister; the Runaway sister and the sister in Denial. I recognized the Abusive Husband and why he was there and then the Father, oh oh the Father... all superbly played. I also recognized myself and this movie was an eye-opener, a relief, a chance to face my OWN truth and finally doing something about it. I truly hope A Thousand Acres has had the same effect on some others out there.'
Predicted label: 0
